# Level 2 - Week 6 - 04 Agent v1 Blueprint

**Estimated time:** 60-90 minutes

## Learning Objectives

- Define a two-tool plan
- Record steps and outputs
- Validate citations


## Overview

A minimal agent should:

- plan a small sequence
- call 2 tools
- capture all intermediate results
- stop safely

## Suggested Agent v1 workflow

Tools:

- `search` (retrieve chunks)
- `write_answer` (generate grounded answer with citations)

Steps:

1. Plan: decide to search, then answer.
2. Call `search`.
3. Call `write_answer` with the retrieved chunks.
4. Validate citations.
5. Return final.

## Correctness invariants

- Every tool call is recorded in `state.steps` (inputs + outputs + errors).
- The agent never assumes a tool succeeded.
- If the agent returns an answer with citations, citations must be traceable to retrieved chunks.

## Minimal decision rule (stop vs clarify)

Before calling `write_answer`, decide whether retrieval is “good enough”.

- if `hits` is empty → `mode=clarify`
- else → continue to `write_answer`

Later you can incorporate score thresholds (Week 4) and eval-driven tuning (Week 5).

### Sample code

Agent blueprint with search and write_answer.


In [ ]:
AGENT_FLOW = {
    'plan': ['search', 'write_answer'],
    'steps': ['call search', 'call write_answer', 'validate citations', 'return final'],
}

print(AGENT_FLOW)


### Student fill-in

Implement the citation validation rule:

- every `chunk_id` in `citations` must be present in the retrieved evidence set

Then decide your safe fallback when citations are invalid:

- retry once with a repair instruction, or
- switch to `mode=clarify` / `mode=refuse`

In [ ]:
def citations_reference_retrieved(citations: list[dict], retrieved_chunk_ids: set[str]) -> bool:
    for c in citations:
        chunk_id = c.get("chunk_id")
        if not chunk_id or chunk_id not in retrieved_chunk_ids:
            return False
    return True


retrieved = {"kb#001", "kb#002"}

citations_ok = [{"chunk_id": "kb#001", "snippet": "..."}]
citations_bad = [{"chunk_id": "kb#999", "snippet": "..."}]

print("ok:", citations_reference_retrieved(citations_ok, retrieved))
print("bad:", citations_reference_retrieved(citations_bad, retrieved))

## Self-check

- Is the plan short and explicit?
- Are steps logged for debugging?
